In [1]:
import os
import logging
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from dask.distributed import Client

# Set global variables
PROJECT = r"D:\World Bank\Paper - Child mortality and Climate Shocks"
OUTPUTS = rf"{PROJECT}\Outputs"
DATA = rf"{PROJECT}\Data"
DATA_IN = rf"{DATA}\Data_in"
DATA_PROC = rf"{DATA}\Data_proc"
DATA_OUT = rf"{DATA}\Data_out"
ERA5_DATA = r"D:\Datasets\ERA5 Reanalysis\monthly-single-levels"

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 31.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62632,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 31.88 GiB
Comm: tcp://127.0.0.1:62660,Total threads: 1
Dashboard: http://127.0.0.1:62661/status,Memory: 7.97 GiB
Nanny: tcp://127.0.0.1:62635,


2025-04-25 23:52:15,873 - distributed.scheduler - WARNING - Worker failed to heartbeat for 82523s; attempting restart: <WorkerState 'tcp://127.0.0.1:62649', name: 2, status: running, memory: 0, processing: 0>
2025-04-25 23:52:17,920 - distributed.scheduler - WARNING - Worker failed to heartbeat for 82523s; attempting restart: <WorkerState 'tcp://127.0.0.1:62652', name: 1, status: running, memory: 0, processing: 0>
2025-04-25 23:52:17,926 - distributed.scheduler - WARNING - Worker failed to heartbeat for 82522s; attempting restart: <WorkerState 'tcp://127.0.0.1:62653', name: 3, status: running, memory: 0, processing: 0>
2025-04-25 23:52:17,930 - distributed.scheduler - WARNING - Worker failed to heartbeat for 82523s; attempting restart: <WorkerState 'tcp://127.0.0.1:62660', name: 0, status: running, memory: 0, processing: 0>
2025-04-25 23:52:23,068 - distributed.nanny - WARNING - Restarting worker
2025-04-25 23:52:23,455 - distributed.nanny - WARNING - Restarting worker
2025-04-25 23:52

In [3]:
climate_shocks_v9 = xr.open_dataset(
        r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\Climate_shocks_v9.nc",
        chunks={"lat": 60, "lon": 60, "time": 5}
    ).drop_vars("month")
lat_tgt = climate_shocks_v9.lat
lon_tgt = climate_shocks_v9.lon

# Interpolate and store the datasets
tasks = []
for var in ["hd35", "hd40", "fd", "id"]:
    ds_var = xr.open_dataset(
        rf"{DATA_IN}\Climate Data\timeseries-{var}-monthly-mean_era_monthly_era5-0.5x0.5-timeseries_mean_1950-2020.nc",
        chunks={"lat": 60, "lon": 60, "time": 5})\
        .sel(bnds=0, time=slice("1990-01-01", "2020-12-01"))\
        .drop_vars("bnds")\
        [f"timeseries-{var}-monthly-mean"].rename(var)
    ds_var = (ds_var / np.timedelta64(1, 'D')).astype(int)
    # ds_var = ds_var.transpose("lat", "lon", "time")
    
    ds_var = ds_var.interp(
        lat=lat_tgt,
        lon=lon_tgt,
        method="linear",
    )

    # Save
    outname = rf"{DATA_PROC}\{var}_interpolated.nc"
    ds_var.to_netcdf(outname, encoding={var: {"zlib": True, "complevel": 6}}, compute=True)
    print(f"Saved {var} to {outname}")


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 60. This could degrade perfo

Saved hd35 to D:\World Bank\Paper - Child mortality and Climate Shocks\Data\Data_proc\hd35_interpolated.nc


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Saved hd40 to D:\World Bank\Paper - Child mortality and Climate Shocks\Data\Data_proc\hd40_interpolated.nc


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Saved fd to D:\World Bank\Paper - Child mortality and Climate Shocks\Data\Data_proc\fd_interpolated.nc


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 60. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Saved id to D:\World Bank\Paper - Child mortality and Climate Shocks\Data\Data_proc\id_interpolated.nc


In [4]:
climate_shocks_v9 = xr.open_dataset(
        r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\Climate_shocks_v9.nc",
        chunks={"lat": 200, "lon": 360, "time": None}
    ).drop_vars("month")

# Load the datasets and combine them
dss = [climate_shocks_v9]
for var in ["hd35", "hd40", "fd", "id"]:
    ds_var = xr.open_dataset(
        rf"{DATA_PROC}\{var}_interpolated.nc",
        chunks={"lat": 200, "lon": 360, "time": None}
    )
    dss += [ds_var]

ds = xr.combine_by_coords(dss)

# Export
out = rf"{DATA_PROC}/Climate_shocks_v9d.nc"
encoding = {
    var: {"zlib": True, "complevel": 6} for var in ds.data_vars
}
ds.to_netcdf(
    out,
    encoding=encoding,
    
)


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 360. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 200. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xarray\core\dataset.py:282: UserWarning: The specified chunks separate the 

In [6]:
climate_shocks_v9

<xarray.Dataset> Size: 25GB
Dimensions:    (lon: 1440, lat: 721, time: 360)
Coordinates:
  * lon        (lon) float64 12kB -179.8 -179.5 -179.2 ... 179.5 179.8 180.0
  * lat        (lat) float32 3kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * time       (time) datetime64[ns] 3kB 1991-01-01 1991-02-01 ... 2020-12-01
Data variables:
    absdif_t   (time, lat, lon) float64 3GB dask.array<chunksize=(60, 200, 360), meta=np.ndarray>
    absdifm_t  (time, lat, lon) float64 3GB dask.array<chunksize=(60, 200, 360), meta=np.ndarray>
    spi1       (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    spi12      (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    spi24      (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    spi3       (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    spi48      (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    spi6       (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    spi9       (time, lat, lon) float32 1GB dask.array<chunksize=(72, 200, 360), meta=np.ndarray>
    std_t      (time, lat, lon) float64 3GB dask.array<chunksize=(60, 200, 360), meta=np.ndarray>
    stdm_t     (time, lat, lon) float64 3GB dask.array<chunksize=(60, 200, 360), meta=np.ndarray>
    t          (time, lat, lon) float64 3GB dask.array<chunksize=(60, 200, 360), meta=np.ndarray>